# Generación de Texto en Español Imitando el Estilo de Ibai Llanos con RNN y Keras

**Autoras:** Alina Rojas y Mar Iborra<br>
**Basado en el tutorial de TensorFlow:** [Generación de texto](https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419)<br>
**Fuente del Dataset:** [Aprende Machine Learning](https://www.aprendemachinelearning.com/) - Dataset de Ibai Llanos<br>
**Fecha de creación de la adaptación:** 2023/12/31<br>
**Última modificación:** 2024/01/12<br>
**Descripción:** Este proyecto se centra en el desarrollo de un modelo de Red Neuronal Recurrente (RNN) utilizando Keras, con el objetivo de generar texto en español que imite el estilo de comunicación del famoso streamer Ibai Llanos. Inspirado en la tutorial de generación de texto de TensorFlow, este modelo se entrena con transcripciones de diálogos y entrevistas de Ibai, proporcionados por Aprende Machine Learning. El propósito es explorar las capacidades de las RNN en la generación de texto que refleje el lenguaje natural y el estilo único de un personaje público conocido.

## Introducción

En este proyecto, nos centraremos en la generación de texto en español utilizando Redes Neuronales Recurrentes (RNN) con Keras, inspirándonos en la técnica y el estilo de comunicación del conocido streamer Ibai Llanos.

Este trabajo se basa en el tutorial de generación de texto de TensorFlow, disponible en [TensorFlow Tutorials](https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419), y utiliza un dataset específico de Ibai Llanos proporcionado por [Aprende Machine Learning](https://www.aprendemachinelearning.com/). A diferencia del tutorial original, que se enfoca en la generación de texto en inglés, nuestro proyese adaptaráicas para trabajar con texto en español, reflejas y el estilo único de Ibai Llanos.

Los objetivos y pasos clave de este proyecto incluyen:

- Preprocesamiento y tokenización del texto en español utilizando herramientas adecuadas para mantener la integridad del lenguaje y estilo.
- Construcción y entrenamiento de un modelo RNN con Keras, ajustado para capturar y replicar el estilo de Ibai Llanos en la generación de texto.
- Implementación de técnicas de generaciónpara intentar que elermitan pueda al modelo producir respuestas coherentes y creativas en español, utilizando las características lingüísticas y de estilo del dataset.

Nuestro enfoque permitirá explorar las capacidades de las RNN en la generación de texto en un idioma diferente al inglés y en capturar el estilo de una personalidad específica, ampliando así el alcance y la aplicabilidad de los modelos de generación de texto en el campo del procesamiento del lenguaje natural.

## Configuración

##### Importar TensorFlow y otras bibliotecas.

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os

##### Lectura de ficheros de datos

In [2]:
datos_ibai = "transcripcionesIbai.txt"

##### Leer los datos

In [3]:
# Leer el archivo y luego decodificarlo.
text = open(datos_ibai, 'rb').read().decode(encoding='utf-8')
# La longitud del texto es el número de caracteres que contiene
print(f'Longitud del texto: {len(text)} caracteres')

Longitud del texto: 2139034 caracteres


In [4]:
# Echar un vistazo a los primeros 250 caracteres del texto
print(text[:250])

 hola, buenas tardes.  por fin.  hola.  creo que estoy  sí, te escucho, pero a mí me está dando  ¿ me escuchas ? estoy muy nervioso, eh. 
 perdóname, dios.  me acabo de poner mazo de nervioso, tío.  un segundo, ¿ eh ? vale.  ¿ qué tal ? ¿ todo bien 


In [5]:
# Los caracteres únicos en el archivo
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

114 caracteres únicos


## Procesar el texto

##### Vectorizar el texto


Antes del entrenamiento, debe convertir las cadenas en una representación numérica. <br>
La capa `tf.keras.layers.StringLookup` puede convertir cada carácter en un ID numérico. Solo necesita que el texto se divida en tokens primero.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Ahora creamos la capa `tf.keras.layers.StringLookup` :

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[28, 29, 30, 31, 32, 33, 34], [51, 52, 53]]>

Dado que el objetivo de este tutorial es generar texto, también será importante invertir esta representación y recuperar cadenas legibles por humanos a partir de ella. Para esto, puede usar `tf.keras.layers.StringLookup(..., invert=True)`.

Nota: Aquí, en lugar de pasar el vocabulario original generado con `sorted(set(text))` use el método `get_vocabulary()` de la capa `tf.keras.layers.StringLookup` para que los tokens `[UNK]` se configuren de la misma manera.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Esta capa recupera los caracteres de los vectores de ID y los devuelve como un `tf.RaggedTensor` de caracteres:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Puede `tf.strings.reduce_join` para volver a unir los caracteres en cadenas.

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## La tarea de predicción

Dado un carácter, o una secuencia de caracteres, ¿cuál es el próximo carácter más probable? Esta es la tarea para la que está entrenando al modelo. La entrada al modelo será una secuencia de caracteres, y usted entrena el modelo para predecir la salida: el siguiente carácter en cada paso de tiempo.

Dado que los RNN mantienen un estado interno que depende de los elementos vistos anteriormente, dados todos los caracteres computados hasta este momento, ¿cuál es el siguiente carácter?

#### 1. Crear ejemplos de entrenamiento y objetivos

A continuación, dividimos el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá caracteres `seq_length` del texto.

Para cada secuencia de entrada, los objetivos correspondientes contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha.

Así que divide el texto en partes de `seq_length+1` . Por ejemplo, digamos que `seq_length` es 4 y nuestro texto es "Hola". La secuencia de entrada sería "Hola" y la secuencia objetivo "ola".

Para hacer esto, primero utiliza la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en un flujo de índices de carcteres.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(2139034,), dtype=int64, numpy=array([ 3, 35, 42, ..., 42, 14,  3])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

 
h
o
l
a
,
 
b
u
e


In [16]:
seq_length = 100

A continuación vamos a procesar el `Dataset` de IDs de caracteres para formar secuencias de entrenamiento para el modelo.

1. `sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)`: Esta línea agrupa los IDs en lotes, cada uno de tamaño `seq_length+1`. El argumento `drop_remainder=True` asegura que todos los lotes tengan exactamente este tamaño, descartando cualquier lote más pequeño al final del conjunto de datos.

2. El bucle `for` con `sequences.take(1)` muestra un ejemplo de estas secuencias. Al imprimir `chars_from_ids(seq)`, estamos convirtiendo una secuencia de IDs nuevamente en caracteres para visualizar cómo se ve una secuencia de entrenamiento típica.

Este paso es crucial para estructurar los datos de manera que el modelo de aprendizaje automático pueda aprender a predecir el siguiente carácter en una secuencia de un tamaño fijo, en este caso, `seq_length+1`. Esta estructuración es un paso preparatorio estándar en la creación de modelos de procesamiento de lenguaje natural.


In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b' ' b'h' b'o' b'l' b'a' b',' b' ' b'b' b'u' b'e' b'n' b'a' b's' b' '
 b't' b'a' b'r' b'd' b'e' b's' b'.' b' ' b' ' b'p' b'o' b'r' b' ' b'f'
 b'i' b'n' b'.' b' ' b' ' b'h' b'o' b'l' b'a' b'.' b' ' b' ' b'c' b'r'
 b'e' b'o' b' ' b'q' b'u' b'e' b' ' b'e' b's' b't' b'o' b'y' b' ' b' '
 b's' b'\xc3\xad' b',' b' ' b't' b'e' b' ' b'e' b's' b'c' b'u' b'c' b'h'
 b'o' b',' b' ' b'p' b'e' b'r' b'o' b' ' b'a' b' ' b'm' b'\xc3\xad' b' '
 b'm' b'e' b' ' b'e' b's' b't' b'\xc3\xa1' b' ' b'd' b'a' b'n' b'd' b'o'
 b' ' b' ' b'\xc2\xbf' b' ' b'm' b'e'], shape=(101,), dtype=string)


Vamos a iterar sobre las primeras cinco secuencias del `Dataset` de TensorFlow y visualizándolas:

1. El bucle `for` con `sequences.take(5)` selecciona las primeras cinco secuencias del conjunto de datos.

2. `print(text_from_ids(seq).numpy())`: Dentro del bucle, convertimos cada secuencia de IDs numéricos de vuelta a texto usando la función `text_from_ids` y luego lo convertimos a un arreglo NumPy para imprimirlo.

Este proceso nos permite ver cómo se han transformado las secuencias de texto originales en secuencias de IDs y luego de vuelta a texto. Es una forma de verificar que la conversión de datos se ha realizado correctamente y de entender mejor cómo se están preparando los datos para el entrenamiento del modelo. Esta visualización es una práctica común para asegurarse de que los datos se procesan como se espera antes de proceder con el entrenamiento del modelo.


In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b' hola, buenas tardes.  por fin.  hola.  creo que estoy  s\xc3\xad, te escucho, pero a m\xc3\xad me est\xc3\xa1 dando  \xc2\xbf me'
b' escuchas ? estoy muy nervioso, eh. \r\n perd\xc3\xb3name, dios.  me acabo de poner mazo de nervioso, t\xc3\xado.  un'
b' segundo, \xc2\xbf eh ? vale.  \xc2\xbf qu\xc3\xa9 tal ? \xc2\xbf todo bien ? todo bien. \r\n aqu\xc3\xad.  viendo que equipo va a coger, '
b'no s\xc3\xa9 qu\xc3\xa9 equipo coge.  no s\xc3\xa9 si estoy contigo, conectado, no s\xc3\xa9 qu\xc3\xa9 es lo que  s\xc3\xad, s\xc3\xad, est\xc3\xa1s conmigo'
b'.  el otro de la partida soy yo. \r\n el otro de la partida soy yo, s\xc3\xad.  ok, ya, ya.  bueno, como lo pr'


Para el entrenamiento, necesitará un conjunto de datos de (input, label) pares. Donde input y label son secuencias. En cada paso de tiempo, la entrada es el carácter actual y la etiqueta es el siguiente carácter.

Aquí hay una función que toma una secuencia como entrada, la duplica y la cambia para alinear la entrada y la etiqueta para cada paso de tiempo:

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b' hola, buenas tardes.  por fin.  hola.  creo que estoy  s\xc3\xad, te escucho, pero a m\xc3\xad me est\xc3\xa1 dando  \xc2\xbf m'
Target: b'hola, buenas tardes.  por fin.  hola.  creo que estoy  s\xc3\xad, te escucho, pero a m\xc3\xad me est\xc3\xa1 dando  \xc2\xbf me'


#### 2. Crear lotes de entrenamiento

Usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, debe mezclar los datos y empaquetarlos en lotes.

In [23]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#### 3. Build The Model

Los siguientes fragmentos de código define una clase de modelo personalizado utilizando TensorFlow Keras para un modelo de generación de texto:

1. Parámetros del Modelo:
   - `vocab_size = len(ids_from_chars.get_vocabulary())`: Define el tamaño del vocabulario, que es el número de caracteres únicos en el texto.
   - `embedding_dim = 256`: Establece la dimensión de incrustación (embedding) en 256. Las incrustaciones son representaciones densas de bajo dimensionalidad de cada carácter.
   - `rnn_units = 1024`: Especifica el número de unidades en la capa de Red Neuronal Recurrente (RNN).

2. Clase `MyModel`:
   - La clase `MyModel` hereda de `tf.keras.Model` y define un modelo de red neuronal.
   - La capa `Embedding` transforma los índices de caracteres en vectores densos de tamaño `embedding_dim`.
   - La capa `GRU` (una forma de RNN) procesa secuencias de vectores de incrustación y mantiene un estado interno que captura la información de los caracteres anteriores.
   - La capa `Dense` convierte la salida de la RNN en probabilidades para cada carácter en el vocabulario.
   - La función `call` define cómo pasa la entrada a través de estas capas. Si `return_state` es verdadero, devuelve tanto la salida como el estado interno, útil para generación de texto.

3. Instanciación del Modelo:
   - `model = MyModel(vocab_size, embedding_dim, rnn_units)`: Crea una instancia del modelo con los parámetros de vocabulario, dimensión de incrustación y unidades RNN especificados.

Este modelo es adecuado para tareas de generación de texto, donde el objetivo es predecir el siguiente carácter en una secuencia basándose en los caracteres anteriores.


In [24]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### 4. Probando el modelo

El siguiente código demuestra cómo realizar una predicción con un lote de datos utilizando el modelo definido previamente:

1. El bucle `for` recorre un solo lote del conjunto de datos `dataset`. Este lote contiene pares de secuencias de entrada (`input_example_batch`) y sus correspondientes secuencias objetivo (`target_example_batch`).

2. `example_batch_predictions = model(input_example_batch)`: Aquí, el modelo realiza predicciones en el lote de entradas. El modelo devuelve la salida para cada secuencia de entrada en el lote.

3. `print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")`: Imprime la forma de las predicciones. La salida tiene tres dimensiones: tamaño del lote (`batch_size`), longitud de la secuencia (`sequence_length`) y tamaño del vocabulario (`vocab_size`). Esta forma indica que, para cada carácter en cada secuencia del lote, el modelo ha generado un vector de probabilidad sobre el vocabulario entero, representando la probabilidad de cada carácter de ser el siguiente en la secuencia.

Este paso es crucial para verificar la estructura de salida del modelo y asegurarse de que está generando predicciones en la forma esperada, lo cual es fundamental para proceder con el entrenamiento y la evaluación del modelo.


In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 115) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  29440     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  117875    
                                                                 
Total params: 4085619 (15.59 MB)
Trainable params: 4085619 (15.59 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


A continuación vamos a seleccionar caracteres al azar de las predicciones del modelo para una secuencia de ejemplo:

1. `sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)`: Utiliza la función `tf.random.categorical` para muestrear índices de caracteres basándose en las distribuciones de probabilidad proporcionadas por `example_batch_predictions[0]`, que representa las predicciones del modelo para el primer elemento en el lote. El parámetro `num_samples=1` indica que se seleccionará un carácter por cada paso de tiempo en la secuencia.

2. `sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()`: La función `tf.squeeze` elimina las dimensiones de tamaño 1 del tensor `sampled_indices`, simplificando su estructura. Luego, se convierte el tensor resultante en un arreglo NumPy.

Estos pasos son parte del proceso de generar texto con el modelo: basándose en las predicciones del modelo, seleccionamos al azar el siguiente carácter en cada paso de tiempo, lo que nos permite construir una secuencia de caracteres generada por el modelo.


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

El siguiente código nos dará, en cada paso de tiempo, una predicción del siguiente índice de caracteres:

In [30]:
sampled_indices

array([ 23,  78, 109,  93,  63,  66,  23,  88,  78,   2,  24,  43,  69,
       104, 114,  94,  51,  81,  59,  76,   1,  91,   5,  93,  35,  99,
        22,  41,  54,  86,   0,  50,  95,  18,  72, 111,  32,  95,  71,
        26,  84,  28, 108, 112,  13,  84,  68,  25,  74, 101,  13,  42,
        13,  97,  30,  51,   5,  22,   3,  62,  83, 101,  97,  67,  15,
        55,  34,  95,  31,  76,   4,  47,  10,  84,  95,  16,  68,  82,
        74,  11,  69, 104,  44,  32,  96,  66,  68,  71, 104,  35,  43,
       104,  92,  39,  70, 110,  42,  18,  68,  88])

Decodificamos estos para ver el texto predicho por este modelo no entrenado:

In [31]:
print("Entrada:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Predicciones del Siguiente Carácter:\n", text_from_ids(sampled_indices).numpy())

Entrada:
 b' \r\n o sea, ir por una nota muy grave y de la nada pasar a una muy aguda, cosa que hice en colocao va'

Predicciones del Siguiente Carácter:
 b'8\xd0\xb2\xec\x98\xa4\xe5\x80\x91\xc3\xa5\xc3\xaa8\xe3\x83\x88\xd0\xb2\r9p\xc3\xb2\xeb\xa8\xb9\xf0\x9f\x8e\xb5\xe6\x83\xb4x\xd0\xb6\xc2\xbf\xcf\x87\n\xe3\x83\xbc"\xe5\x80\x91h\xea\xb0\x997n|\xd1\x87[UNK]w\xe6\xae\x983\xc3\xb6\xec\xb6\xa4e\xe6\xae\x98\xc3\xb4>\xd1\x80a\xec\x96\xb4\xed\x99\x8d-\xd1\x80\xc3\xb1<\xc3\xbc\xeb\x86\x93-o-\xe8\xb4\x8fcx"7 \xc3\xa4\xd0\xbf\xeb\x86\x93\xe8\xb4\x8f\xc3\xad0\xc2\xa1g\xe6\xae\x98d\xcf\x87!t*\xd1\x80\xe6\xae\x981\xc3\xb1\xd0\xb7\xc3\xbc+\xc3\xb2\xeb\xa8\xb9qe\xe7\x97\x85\xc3\xaa\xc3\xb1\xc3\xb4\xeb\xa8\xb9hp\xeb\xa8\xb9\xe4\xbb\xacl\xc3\xb3\xec\x9b\xa0o3\xc3\xb1\xe3\x83\x88'


Basado en los resultados proporcionados, aquí tienes un comentario adaptado:
En esta parte del notebook, estamos observando la decodificación y visualización de la salida de un modelo de lenguaje que aún no ha sido entrenado. Esta práctica es crucial para entender cómo el modelo realiza predicciones antes de ser sometido a un proceso de aprendizaje. Analicemos en detalle lo que está sucediendo:

1. **Entrada Decodificada**:
   - La entrada (`Entrada`) muestra una parte de un texto que parece ser una conversación o un comentario sobre música, evidenciado por la referencia a notas musicales graves y agudas.
   - La cadena de entrada es: `" o sea, ir por una nota muy grave y de la nada pasar a una muy aguda, cosa que hice en colocao va"`.

2. **Predicciones de Siguiente Carácter**:
   - Las `Predicciones del Siguiente Carácter` son las estimaciones del modelo sobre qué caracteres podrían seguir a la entrada proporcionada.
   - Dado que el modelo aún no está entrenado, estas predicciones son fundamentalmente aleatorias, resultando en una secuencia sin sentido compuesta por una mezcla de caracteres, incluyendo letras, números, símbolos y caracteres especiales.

**Análisis**:
- Esta salida ilustra el comportamiento de un modelo de lenguaje no entrenado, donde realiza predicciones basadas en la aleatoriedad en lugar de en el conocimiento del lenguaje. La falta de coherencia y sentido en las predicciones refleja la ausencia de aprendizaje previo del modelo.
- Esta etapa es ideal para comprender el estado inicial del modelo, que aún no ha adquirido la capacidad de entender o generar texto con estructura gramatical o sintáctica coherente.

**Conclusión**:
- Observar las predicciones de un modelo de lenguaje no entrenado es útil para establecer una base antes de su entrenamiento. Permite apreciar el impacto y la importancia del entrenamiento en la evolución del modelo hacia la generación de texto coherente y contextualmente adecuado.
- Este proceso destaca cómo el entrenamiento transforma un modelo de generar respuestas aleatorias a producir texto que es gramaticalmente correcto y contextualmente relevante, demostrando la importancia del entrenamiento en el desarrollo de modelos de lenguaje efectivos.

#### 5. Entrenando el modelo

##### Adjuntamos un optimizador y una función de pérdida

Vamos a calcular la pérdida (loss) media para un lote de ejemplos, un paso crucial en el entrenamiento y la evaluación de modelos de aprendizaje automático:

1. `loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)`: Define la función de pérdida como entropía cruzada categórica dispersa. El parámetro `from_logits=True` indica que la función debe aplicarse a las salidas del modelo que son logits (valores previos a la aplicación de una función de activación como softmax). Esta función de pérdida es adecuada para problemas de clasificación multiclase como la generación de texto, donde cada carácter generado se trata como una clase distinta.

2. `example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)`: Calcula la pérdida media para un lote de ejemplos comparando las predicciones del modelo (`example_batch_predictions`) con las secuencias objetivo reales (`target_example_batch`). La función de pérdida evalúa qué tan bien las predicciones del modelo coinciden con los objetivos reales.

3. Las líneas `print` muestran la forma de las predicciones y la pérdida media. La forma de las predicciones confirma que el modelo está generando salidas en el formato esperado (`batch_size`, `sequence_length`, `vocab_size`). La pérdida media proporciona una medida de cuán precisas son las predicciones del modelo: una pérdida baja indica predicciones más precisas.

Calcular y monitorear la pérdida media es esencial para entender el rendimiento del modelo y para ajustar los parámetros del modelo o el proceso de entrenamiento con el fin de mejorar la precisión de las predicciones.


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Forma de la predicción: ", example_batch_predictions.shape, " # (tamaño del lote, longitud de la secuencia, tamaño del vocabulario)")
print("Pérdida media:        ", example_batch_mean_loss)

Forma de la predicción:  (64, 100, 115)  # (tamaño del lote, longitud de la secuencia, tamaño del vocabulario)
Pérdida media:         tf.Tensor(4.7424912, shape=(), dtype=float32)


En la siguiente línea de código se calcula y se muestra el valor exponencial de la pérdida media calculada previamente para el lote de ejemplos:

- `tf.exp(example_batch_mean_loss).numpy()`: Aplica la función exponencial (`tf.exp`) a la pérdida media (`example_batch_mean_loss`) y convierte el resultado a un arreglo NumPy. Este cálculo transforma la pérdida media, que es una medida de error, en una métrica más interpretable. En el contexto del aprendizaje automático, especialmente en problemas de clasificación como la generación de texto, este valor puede proporcionar una indicación más intuitiva de la efectividad del modelo.


In [34]:
tf.exp(example_batch_mean_loss).numpy()

114.71965

Compilamos el modelo, especificando el optimizador y la función de pérdida:

- `model.compile(optimizer='adam', loss=loss)`: Configura el modelo para el entrenamiento. Se utiliza el optimizador 'adam', un algoritmo de optimización popular para entrenar redes neuronales, y se establece la función de pérdida definida previamente (`loss`). La compilación es un paso esencial antes de comenzar el entrenamiento, ya que establece cómo el modelo debe actualizarse y evaluarse.

In [35]:
model.compile(optimizer='adam', loss=loss)

##### Configurar checkpoints

In [36]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

##### Ejecutamos el entrenamiento

In [37]:
EPOCHS = 30

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
330/330 [==============================] - 25s 62ms/step - loss: 2.2365
Epoch 2/30
330/330 [==============================] - 22s 61ms/step - loss: 1.5936
Epoch 3/30
330/330 [==============================] - 21s 60ms/step - loss: 1.3721
Epoch 4/30
330/330 [==============================] - 21s 60ms/step - loss: 1.2730
Epoch 5/30
330/330 [==============================] - 22s 60ms/step - loss: 1.2138
Epoch 6/30
330/330 [==============================] - 22s 62ms/step - loss: 1.1694
Epoch 7/30
330/330 [==============================] - 22s 59ms/step - loss: 1.1316
Epoch 8/30
330/330 [==============================] - 21s 59ms/step - loss: 1.0984
Epoch 9/30
330/330 [==============================] - 21s 60ms/step - loss: 1.0665
Epoch 10/30
330/330 [==============================] - 22s 60ms/step - loss: 1.0346
Epoch 11/30
330/330 [==============================] - 21s 59ms/step - loss: 1.0029
Epoch 12/30
330/330 [==============================] - 22s 59ms/step - loss: 0.9717
E

El proceso de entrenamiento de nuestro modelo RNN a lo largo de estas 30 épocas ha mostrado una notable eficiencia en comparación con la práctica anterior, donde se necesitaban al menos 300 épocas para lograr un texto coherente. Analicemos los resultados obtenidos y las posibles razones de esta mejora:

- **Inicio del Entrenamiento**: Durante las primeras épocas, se observa una rápida disminución en la pérdida (`loss`), pasando de 2.2365 a 1.5936. Este descenso rápido es indicativo de que el modelo está captando las estructuras básicas del lenguaje de manera eficiente, posiblemente debido a la amplia variedad y riqueza del nuevo dataset.

- **Progresión Continua de la Pérdida**: A medida que avanzan las épocas, la pérdida sigue disminuyendo, aunque a un ritmo más gradual. Esto sugiere que el modelo está afinando su aprendizaje, adaptándose a las complejidades y matices del texto. La pérdida se reduce de manera constante, pasando por valores como 1.2730, 1.0984 y 0.9717, lo que demuestra un proceso de aprendizaje sostenido.

- **Estabilización hacia el Final del Entrenamiento**: En las últimas épocas, la pérdida se estabiliza alrededor de 0.7189, mostrando fluctuaciones menores. Esta estabilización indica que el modelo ha alcanzado un equilibrio en su aprendizaje, absorbiendo eficazmente la información del dataset sin sobreajustarse.

**Comparación con la Práctica 2**:
- Es notable que, en comparación con la práctica 2 donde se necesitaban al menos 300 épocas, en esta ocasión solo hemos requerido 30 épocas para obtener resultados coherentes. Esto se puede atribuir a la mayor extensión y diversidad del nuevo dataset utilizado, que ha proporcionado una base de datos más rica y variada para el entrenamiento.

**Conclusión**:
- La eficiencia del entrenamiento en este proyecto demuestra cómo un dataset extenso y diverso puede mejorar significativamente la capacidad de aprendizaje de un modelo de lenguaje. La disminución constante y la estabilización de la pérdida sugieren que el modelo ha logrado captar la esencia del lenguaje y estilo de Ibai Llanos, lo que se espera se refleje en la calidad del texto generado.

#### 6. Generación texto y evaluación de su calidad

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.8):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Crear una máscara para evitar que se genere "[UNK]".
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Poner un -inf en cada índice no deseado.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Ajustar la forma al vocabulario
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convertir cadenas de texto en IDs de tokens.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Ejecutar el modelo.
    # La forma de predicted_logits es [lote, carácter, logits_del_siguiente_carácter]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Utilizar solo la última predicción.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Aplicar la máscara de predicción: evitar que se genere "[UNK]".
    predicted_logits = predicted_logits + self.prediction_mask

    # Muestrear los logits de salida para generar IDs de tokens.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convertir de IDs de tokens a caracteres
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Devolver los caracteres y el estado del modelo.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
start = time.time()
states = None
next_char = tf.constant(['¿ que no hay'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

¿ que no hay que hacerla ? pues ya te digo. 
 ayer lo tengo yo también.  una setup privada siempre.  hace años ya.  ¿ qué es esto ? esto no es booque townetó, coño.  es que es como uno de los shows gorno.  se ha de bates el mejor. 
 mira.  voy a decir dos cosas.  vences entendrás que estabas repartido como el mes mío, vamos a hacer un tema cuando lo hacía con su favorito sería una carencia de que hablaba es como yo mí me gustaba encontrara que se llevando covid.  y no lo pienso, no sé qué ha pasado, ¿ no ? ¿ quieres escuchar este año ? ¿ de quiénes se seguías ? que lo grabé una barbarsica para que geste nunca tengo el visa y el resto va a foca 300.  ya he visto el castel.  me he caído.  es cara esta, está lleno de coimbra.  la verdad es que no sé qué decía k, soy yo calla, hijo del granjero, te lo digo, como el de la bandera correcta de puerto rico, que al final es lo que hablamos y están ahí escuchando una cosa.  hemos hecho muchas cosas como son. 
 es como un sebado bastante noble, l

Este texto generado por nuestro modelo RNN, entrenado para imitar el estilo comunicativo de Ibai Llanos, refleja una mezcla de coherencia y espontaneidad característica del influencer. A continuación, destaco algunos aspectos importantes del resultado:

- **Estilo Informal y Conversacional**: El texto muestra un estilo informal y conversacional, con frases típicas de Ibai como "¿que no hay que hacerla? pues ya te digo" o "ayer lo tengo yo también". Esto sugiere que el modelo ha captado con éxito el tono coloquial y directo de Ibai.

- **Elementos de Cultura Popular y Referencias Personales**: Se observan numerosas referencias a temas de actualidad, cultura popular y experiencias personales, como "setup privada", "shows gorno", "covid", o "el de la bandera correcta de Puerto Rico". Esto indica que el modelo ha aprendido a incorporar elementos que reflejan los intereses y el estilo de vida de Ibai.

- **Fluidez y Repeticiones**: Aunque hay segmentos del texto que fluyen de manera coherente, también hay repeticiones y frases fragmentadas, como "es cara esta, está lleno de Coimbra" o "el chico está esperando". Estas incoherencias reflejan las limitaciones del modelo para mantener una narrativa completamente lógica y coherente.

- **Comparación con Prácticas Anteriores**: En comparación con la práctica anterior, donde se necesitaban al menos 300 épocas para lograr coherencia, este modelo ha requerido solo 30 épocas para producir resultados significativos. Esto se debe probablemente a la mayor diversidad y extensión del dataset actual.

**Conclusión**:
- Los resultados muestran un modelo que ha aprendido a capturar varios aspectos del estilo y lenguaje de Ibai Llanos, aunque aún enfrenta desafíos en términos de coherencia y consistencia narrativa. La reducción del número de épocas necesarias, en comparación con la práctica anterior, destaca la importancia de un dataset amplio y variado para el entrenamiento eficiente de modelos de lenguaje.

#### 8. Salvamos el modelo

In [42]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [43]:
states = None
next_char = tf.constant(['¿ te acuerdas'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

¿ te acuerdas ? correcto.  la escuchéis.  venga, va, que me pongo más europela de betos y con cristiano tenía la 
